# RMN Baseline

Training an RMN on sessions 105 - 111

In [1]:
import os
import pandas as pd

In [2]:
os.chdir("../../../scripts/assembly/")

In [3]:
from document import *
from subject import subject_keywords
from constant import DOC_PATH

In [5]:
# run several times
os.chdir("../modeling")
from token_mapping import *
from embeddings import *
from helper import load_pickled_object
from rmn import RMN

In [6]:
DOC_SAMPLE_PATH = os.path.join(DOC_PATH, "doc-sample/")
docs_df = load_documents(subject_keywords.keys(), DOC_SAMPLE_PATH)
# N = 50000
# docs_df = docs_df.sample(N)

In [7]:
feature_columns = [
 'speakerid',
 'chamber',
 'state',
 'gender',
 'party',
 'congress',
 'subject']

# make tokenizer and metadata dicts
tokenizer_dict = build_tokenizer_dict(docs_df)
metadata_dict = build_metadata_dict(docs_df, feature_columns)
metadata_dict.keys()

dict_keys(['speakerid', 'chamber', 'state', 'gender', 'party', 'congress', 'subject'])

In [8]:
# Get embeddings
embedding_file = "/home/rocassius/gen-data/tools/embbedding_index_50d"
embeddings_index = load_pickled_object(embedding_file)
embeddings_matrix = build_embedding_matrix(tokenizer_dict['word_index'], embeddings_index).astype('float16')

# RMN Training

In [23]:
rmn = RMN()
rmn.embedding_matrix = embeddings_matrix
rmn.tokenizer_dict = tokenizer_dict
rmn.metadata_dict = metadata_dict
rmn.num_topics = 40
rmn.build_model()

In [24]:
import sys

In [25]:
sys.getsizeof(embeddings_matrix)

33753912

In [26]:
inputs = rmn.prep_inputs(docs_df)
y = inputs[0]

In [27]:
rmn.model.fit(x=inputs, y=y, epochs = 10, batch_size = 50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1997661 samples
Epoch 1/10
1997661/1997661 [==============================] - 146s 73us/sample - loss: 6.4798
Epoch 2/10
1997661/1997661 [==============================] - 141s 70us/sample - loss: 6.3304
Epoch 3/10
1997661/1997661 [==============================] - 145s 73us/sample - loss: 6.2983
Epoch 4/10
1997661/1997661 [==============================] - 148s 74us/sample - loss: 6.2736
Epoch 5/10
1997661/1997661 [==============================] - 145s 73us/sample - loss: 6.2696
Epoch 6/10
1997661/1997661 [==============================] - 140s 70us/sample - loss: 6.2681
Epoch 7/10
1997661/1997661 [==============================] - 142s 71us/sample - loss: 6.2629
Epoch 8/10
1997661/1997661 [==============================] - 142s 71us/sample - loss: 6.2654
Epoch 9/10
1997661/1997661 [==============================] - 143s 71us/sample - loss: 6.2642
Epoch 10/10
1997661/1997661 [======

In [28]:
rmn.save_rmn("second", "/home/rocassius/gen-data/models")